In [ ]:
import os
import pandas as pd

In [ ]:
from {{package_name}} import utils
from {{package_name}}.models_training import utils_models

blabla général sur les frameworks

# Entrainement d'un modèle de détection d'auteur 'out of the box'

On va partir de textes de différents auteurs du XIXème siècle et fabriquer un modèle permettant d'identifier automatiquement l'auteur

In [ ]:
# On charge les textes
data_path = utils.get_data_path()
df_texts = pd.read_csv(os.path.join(data_path, 'texts.csv'), sep=';')

Nous allons tout d'abord couper notre dataset en deux parties, un ensemble d'entrainement et un ensemble de test afin de pouvoir, entraîner et (vous l'avez deviné) tester notre modèle. Notez que nous ne fabriquons pas d'ensemble de validation car nous n'allons pas tuner notre modèle. 

Pour cela, nous allons utiliser les scripts contenus dans le package. Pour couper en deux notre dataset, procédez de la manière suivante :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}/{{package_name}}-scripts/utils`

Appelez le script de découpe du dataset pour réaliser un 'stratified split': 

`python 0_split_train_valid_test.py -f texts.csv --perc_train 0.6 --perc_valid 0.0 --perc_test 0.4`

Si par hasard, vous avez fait une fausse manip et qu'il faut regénérez des fichiers de train/test, rajoutez l'argument `--overwrite` pour écrasez les fichiers erronés

Vous pouvez alors voir que deux fichiers, `texts_train.csv` et `texts_test.csv` ont été créés dans `{{package_name}}/{{package_name}}-data`

Vous pouvez regarder à quoi ils ressemblent en utilisant la cellule ci-dessous pour les charger dans le notebook:

In [ ]:
df_train, metadata_train = utils.read_csv(os.path.join(data_path, 'texts_train.csv'))
df_test, metadata_test = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))

Nous allons maintenant faire une étape de préprocessing sur les textes afin de les normaliser et que le modèle performe mieux. Pour cela, nous allons utiliser le script `1_preprocess_data.py` situé ici : `{{package_name}}/{{package_name}}-scripts`

Pour cela :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}/{{package_name}}-scripts`

Appelez le script de préprocessing: 

`python 1_preprocess_data.py -f texts_train.csv texts_test.csv --input_col text`

Vous pouvez remarquer que nous devons préciser la colonne sur laquelle doit agir le préprocessing. Nous pouvons voir le résultat du preprocessing sur l'ensemble de test. Notez qu'une nouvelle colonne a été ajouté, la colonne 'preprocessed_text' qui contient le texte modifié.

In [ ]:
df_test_preprocessed, metadata_test_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_test_preprocess_P1.csv'))

Le préprocessing appliqué est celui fourni de base par le package mais il est très facile d'ajouter ses propres pipelined de preprocessing . Pour cela, ouvrez le fichier : `template_nlp/template_nlp/preprocessing/preprocess.py`. Dans ce module, vous voyez que l'ensemble des pipelines de preprocessing sont renseignés dans la fonction `get_preprocessors_dict`. C'est ici que vous pouvez rajouter vos pipelines personnalisées. 

On peut également jeter un coup d'oeil à la fonction `preprocess_sentence_P1` qui indique les différentes étapes de préprocessing (pour plus d'informations, il suffit de lire la documentation du package words_n_fun, une librairie facilitant tout le travail de manipulation de texte).

Si vous avez l'oeil, vous pouvez remarquer que les accents n'ont pas été traités correctement pas la pipeline `preprocess_sentence_P1`. On va donc corrigez cela. Pour ce faire, il suffit de rajouter `'remove_accents'` en première étape dans la pipeline de la fonction `preprocess_sentence_P1`. Puis relancez : 

`python 1_preprocess_data.py -f texts_train.csv texts_test.csv --input_col text`

Les accents sont maintenant correctement traités:

In [ ]:
df_test_preprocessed, metadata_test_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_test_preprocess_P1.csv'))

Nous allons maintenant entraîner notre modèle en utilisant le script `2_training.py`

Pour cela :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}/{{package_name}}-scripts`

Appelez le script d'entrainement: 

`python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author`

Cela va appliquer un simple TF-IDF + SVM sur les données

Nous pourrions utiliser ensuite le script `3_predict.py` pour utiliser le modèle mais nous allons procéder autrement pour avoir les résultats directement dans ce notebook. Si vous allez dans `{{package_name}}/{{package_name}}-models/model_tfidf_svm` vous allez voir un dossier du type `model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}` qui contient la sauvegarde du modèle que nous venons d'entraîner. 

Vous pouvez par exemple voir le f1_score sur l'ensemble d'entrainement, la matrice de confusion sur l'ensemble d'entraînement etc. Notez que nous aurions pu spécifier un dataset de validation au script `2_training.py` et nous aurions alors aussi accès à des métriques sur ce dataset. 

Ce dossier contient également les fichiers de sauvegarde du modèle que nous allons charger dans la cellule suivante (en remplacant bien sûr `{YYYY_MM_DD-hh_mm_ss}` par les nombres correspondant

In [ ]:
model, model_conf = utils_models.load_model('model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}')

Voilà, le modèle est chargé, prêt à être utilisé. Ce que nous allons directement faire sur l'ensemble de test chargé plus haut :

In [ ]:
df_test_preprocessed['prediction'] = model.predict(df_test_preprocessed['preprocessed_text'])
print('accuracy :', len(df_test_preprocessed[df_test_preprocessed['prediction']==df_test_preprocessed['author']])/len(df_test_preprocessed))

Mais, il est tout nul ce modèle !!!!

In [ ]:
nb_word_sentence = 60

In [ ]:
# On constitue les phrases
list_phrases = []
for index, row in df_texts.iterrows():
    text = row['text']
    author = row['author']
    book = row['book']
    # Cette fonction transforme un texte en phrases
    sentences = text_to_sentence(text, nb_word_sentence)
    list_phrases = list_phrases+[(sentence, author, book) for sentence in sentences]
df_phrases = pd.DataFrame(list_phrases, columns=['sentence', 'author', 'book'])
set_author = set(df_texts['author'])

# On regarde la répartition des phrases par auteur
df_phrases.value_counts('author')/len(df_phrases)

In [ ]:
# On mélange le dataset
df_to_split = df_phrases.sample(frac=1)

# On sélectionne un livre par auteur qui sera dans l'ensemble de validation
dict_books_to_valid = {}
for author in set_author:
    set_books = set(df_texts[df_texts['author']==author]['book'])
    dict_books_to_valid[author] = random.sample(set_books, k=1)[0]
df_valid = df_to_split[df_to_split['book'].isin(dict_books_to_valid.values())].copy()

# On sélectionne un autre livre par auteur qui sera dans l'ensemble de test
dict_books_to_test = {}
for author in set_author:
    set_books = set(df_texts[df_texts['author']==author]['book'])
    set_books.remove(dict_books_to_valid[author])
    dict_books_to_test[author] = random.sample(set_books, k=1)[0]
df_test = df_to_split[df_to_split['book'].isin(dict_books_to_test.values())].copy()

# Tout le reste est dans l'ensemble d'entrainement
df_train = df_to_split.copy()
df_train = df_train[~df_train['book'].isin(dict_books_to_valid.values())]
df_train = df_train[~df_train['book'].isin(dict_books_to_test.values())]

# On sauvegarde les datasets
utils.to_csv(df_train, os.path.join(data_path, 'dataset_texts_train.csv'))
utils.to_csv(df_valid, os.path.join(data_path, 'dataset_texts_valid.csv'))
utils.to_csv(df_test, os.path.join(data_path, 'dataset_texts_test.csv'))

# On regarde la répartition des phrases par auteur dans l'ensemble d'entrainement
df_train.value_counts('author')/len(df_train)

In [ ]:
# On fait un léger tuning
dict_result = {}
count =0
for ngram_range in [(1, 1), (1, 2)]:
    for C in [0.1, 0.5, 1, 2]:
        model = ModelTfidfSvm(tfidf_params = {'ngram_range': ngram_range}, svc_params={'C':C})
        model.fit(df_train['sentence'], df_train['author'], x_valid=df_valid['sentence'], y_valid=df_valid['author'])
        df_train['pred'] = model.predict(df_train['sentence'])
        df_valid['pred'] = model.predict(df_valid['sentence'])
        score_train = f1_score(df_train['author'], df_train['pred'], average='macro')
        score_val = f1_score(df_valid['author'], df_valid['pred'], average='macro')
        dict_tmp = {'score_train':round(score_train, 5), 'score_val':round(score_val, 5), 'ngram_range':ngram_range, 'C':C}
        dict_result[count] = dict_tmp.copy()
        count += 1
        print(dict_tmp)

In [ ]:
# On sélectionne le meilleur modèle
model = ModelTfidfSvm(tfidf_params = {'ngram_range': (1, 2)}, svc_params={'C':1})
model.fit(df_train['sentence'], df_train['author'], x_valid=df_valid['sentence'], y_valid=df_valid['author'])

In [ ]:
# On prédit sur le test
df_test['pred'] = model.predict(df_test['sentence'])
score = f1_score(df_test['author'], df_test['pred'], average='macro')
print(score)

In [ ]:
help(predict_author)

In [ ]:
dict_result = {}
# Pour chaque auteur
for author, book in dict_books_to_test.items():
    # Récupère le texte
    row = df_texts[df_texts['book']==book]
    text = row.iloc[0]['text']
    length = len(text)
    # Prédit l'auteur
    prediction, counter, nb_sentences = predict_author(text, model, nb_word_sentence, perc_sample=1.0)
    # Enregistre le résultat
    dict_result[author] = {'prediction': prediction, 'counter': counter, 'length': len(text), 'nb_sentences':nb_sentences}
# Vérifie si le résultat est bon pour tous les auteurs
print({key==value['prediction'] for key, value in dict_result.items()})
# Montre les résultats
dict_result